In [2]:
library(tidyverse)

In [3]:
root_path     <- here::here()
data_path     <- paste0(root_path, "/01_data")
analysis_path <- paste0(root_path, "/02_analysis")

---

### Import relevant IHS report files for 2017 - 2022

Focusing on deferral numbers associated with transplants and other non-preventative medical treatments

In [4]:
setwd(data_path)

In [5]:
for (yr in 2016:2022) {
    folder_path <- glue::glue("raw/08_ihs_unmet_need/FY {yr} Unmet Need Reports")
    
    tertiary_file <- list.files(folder_path, pattern = "Tertiary")
    tertiary_df.raw <- read.csv(paste0(folder_path, "/", tertiary_file), header = F)
    
    tertiary_cols <- t(tertiary_df.raw[7:8,2:37]) %>%
        as_tibble() %>%
        set_names("service_unit", "fed_tribal_total") %>%
        mutate(across(everything(), ~ case_when(.x != "" ~ .x))) %>%
        fill(service_unit) %>%
        mutate(col = toupper(paste0(service_unit, " ~ ", fed_tribal_total))) %>%
        pull(col)
    
    tertiary_df <- tertiary_df.raw[10,2:37] %>%
        set_names(tertiary_cols) %>%
        pivot_longer(everything(), values_to = "N") %>%
        separate(name, into = c("SERVICE_UNIT", "FEDERAL_OR_TRIBAL"), sep = " ~ ") %>%
        mutate(YEAR = yr, CATEGORY = "TRANSPLANT", .before = everything()) %>%
        mutate(FEDERAL_OR_TRIBAL = str_trim(FEDERAL_OR_TRIBAL),
               N = replace_na(suppressWarnings(as.numeric(gsub(",", "", N))), 0))
    
    assign(glue::glue("tertiary_df_{yr}"), tertiary_df)
    
    acute_file <- list.files(folder_path, pattern = "Acute & Chronic")
    acute_df.raw <- read.csv(paste0(folder_path, "/", acute_file), header = F)
    
    acute_cols <- t(acute_df.raw[7:8,3:38]) %>%
        as_tibble() %>%
        set_names("service_unit", "fed_tribal_total") %>%
        mutate(across(everything(), ~ case_when(.x != "" ~ .x))) %>%
        fill(service_unit) %>%
        mutate(col = paste0(service_unit, " ~ ", fed_tribal_total)) %>%
        pull(col)
    
    acute_df <- cbind(toupper(gsub("^[a-o]. ", "", acute_df.raw[30:44, 2])), 
                      acute_df.raw[30:44, 3:38]) %>%
        set_names(c("CATEGORY", acute_cols)) %>%
        pivot_longer(matches(" ~ "), values_to = "N") %>%
        separate(name, into = c("SERVICE_UNIT", "FEDERAL_OR_TRIBAL"), sep = " ~ ") %>%
        mutate(YEAR = yr, .before = everything()) %>%
        mutate(N = replace_na(suppressWarnings(as.numeric(gsub(",", "", N))), 0))
    
    assign(glue::glue("acute_df_{yr}"), acute_df)
}

In [6]:
acute_df <- data.table::rbindlist(lapply(ls(pattern = "acute_df_20"), get))

In [7]:
tertiary_df <- data.table::rbindlist(lapply(ls(pattern = "tertiary_df_20"), get))

In [8]:
deferrals_df <- bind_rows(tertiary_df, acute_df) %>%
    arrange(SERVICE_UNIT, YEAR, CATEGORY, FEDERAL_OR_TRIBAL)

head(deferrals_df)

YEAR,CATEGORY,SERVICE_UNIT,FEDERAL_OR_TRIBAL,N
<int>,<chr>,<chr>,<chr>,<dbl>
2016,CAT SCANS/NUCLEAR MED./MRI,ALASKA,FEDERAL,0
2016,CAT SCANS/NUCLEAR MED./MRI,ALASKA,TOTAL,0
2016,CAT SCANS/NUCLEAR MED./MRI,ALASKA,TRIBAL,0
2016,DERMATOLOGY,ALASKA,FEDERAL,0
2016,DERMATOLOGY,ALASKA,TOTAL,0
2016,DERMATOLOGY,ALASKA,TRIBAL,0


---

### In-story data

> "[T]he IHS paid claims for inpatient substance-use treatment for an average of just 18 patients a year from 2018 to 2022."

See `01_data/raw/08_ihs_unmet_need/IHS Liver Claims 2018-2021.xlsx`.

> "IHS data show the agency deferred nearly 113,000 patient requests to see specialists from 2018 to 2022, including 6,196 requests to see gastroenterologists, who treat liver disease and make referrals to transplant centers."

There were **112,961 reported tertiary deferrals** from 2018 - 2022, including **6,196 reported deferrals for gastroenterologists**.

In [9]:
deferrals_df %>% 
    filter(FEDERAL_OR_TRIBAL == "TOTAL", 
           YEAR %in% 2018:2022,
           CATEGORY != "TRANSPLANT") %>%
    summarise(N = sum(N))

N
<dbl>
112961


In [10]:
deferrals_df %>% 
    filter(FEDERAL_OR_TRIBAL == "TOTAL", 
           YEAR %in% 2018:2022,
           CATEGORY == "GI/GASTROSCOPY") %>%
    summarise(N = sum(N))

N
<dbl>
6196


> "However, agency data shows it paid claims for an average of just 226 patients a year to see hepatologists. Rates were similar for gastroenterologists."

See `01_data/raw/08_ihs_unmet_need/IHS Liver Claims 2018-2021.xlsx`.

> "Claim data shows the agency chipped in very little to the cost of liver transplantation, averaging \\$11,200 per patient."

> "From 2018 to 2022, the agency kicked in funding for just six liver transplants."

See `01_data/raw/08_ihs_unmet_need/IHS Liver Claims 2018-2021.xlsx`.

> "During the same period, it deferred 105 requests for transplants of all organ types"

IHS reported deferring **105 transplants from 2018 - 2022**.

In [11]:
deferrals_df %>% 
    filter(FEDERAL_OR_TRIBAL == "TOTAL", 
           YEAR %in% 2018:2022,
           CATEGORY == "TRANSPLANT") %>%
    summarise(N = sum(N))

N
<dbl>
105
